In [1]:
import mlflow
from mlflow import MlflowClient
from dotenv import dotenv_values

In [2]:
import sys

env_vars = dotenv_values("../.env")
sys.path.append(f"{env_vars['WORKING_DIR']}")
from utils import load_config

config = load_config("../config/development/config.yaml")

From a terminal, in the same repo as your mlruns directory, run:
```
mlflow server --host 127.0.0.1 --port 5000
```

In [3]:
client = MlflowClient(
    tracking_uri=env_vars["MLFLOW_TRACKING_URI"],
)

experiment_name = f"Enefit DataV1 lightgbm HpoConfigV1"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id
runs_df = mlflow.search_runs([experiment_id])
runs_df.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.train_mae,metrics.test_mae,metrics.training_duration,params.num_trees,params.feature_fraction,params.max_depth,params.num_leaves,params.learning_rate,tags.mlflow.source.name,tags.mlflow.source.git.commit,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.log-model.history
0,54560700f31343be81d3065110ba6d87,215806994480999889,FINISHED,mlflow-artifacts:/215806994480999889/54560700f...,2025-05-03 06:53:28.191000+00:00,2025-05-03 06:53:35.628000+00:00,267.450256,276.514219,0.631784,8,0.3962723889353149,6,24,0.09906308908786306,train.py,da8acbc0b8c1c3f7b3fbae9a652bfbd61a8ef941,gabriel,LOCAL,TrialNumber35-20250503T065327UTC,"[{""run_id"": ""54560700f31343be81d3065110ba6d87""..."
1,8c1446a5f1424e06a1febd5163a17330,215806994480999889,FINISHED,mlflow-artifacts:/215806994480999889/8c1446a5f...,2025-05-03 05:51:55.345000+00:00,2025-05-03 05:52:06.041000+00:00,63.773141,103.473742,1.569685,None,None,None,None,None,train.py,da8acbc0b8c1c3f7b3fbae9a652bfbd61a8ef941,gabriel,LOCAL,DefaultParameters-20250503T055155UTC,"[{""run_id"": ""8c1446a5f1424e06a1febd5163a17330""..."
2,6ce5f9152766473186623e9ad218164b,215806994480999889,FINISHED,mlflow-artifacts:/215806994480999889/6ce5f9152...,2025-04-24 14:24:36.886000+00:00,2025-04-24 14:24:42.042000+00:00,267.962692,277.246492,0.709642,8,0.3962723889353149,6,24,0.09906308908786306,train.py,b3988f18ead5caa415f3d9f2930d395ff5f61fef,gabriel,LOCAL,TrialNumber35-20250424T142435UTC,"[{""run_id"": ""6ce5f9152766473186623e9ad218164b""..."


In [4]:
# runs = client.search_runs(experiment_ids=[experiment_id])
# run_params = runs[0].data.params
# run_params

In [5]:
runs_df = mlflow.search_runs(experiment_ids=[experiment_id])
runs_df.columns

Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.train_mae', 'metrics.test_mae',
       'metrics.training_duration', 'params.num_trees',
       'params.feature_fraction', 'params.max_depth', 'params.num_leaves',
       'params.learning_rate', 'tags.mlflow.source.name',
       'tags.mlflow.source.git.commit', 'tags.mlflow.user',
       'tags.mlflow.source.type', 'tags.mlflow.runName',
       'tags.mlflow.log-model.history'],
      dtype='object')

In [7]:
logs_to_check = [
    "metrics.train_mae",
    "metrics.test_mae",
]

seen = set()
i = 0
for _, run in runs_df.iterrows():
    param_values = tuple(run.get(f"{log}", None) for log in logs_to_check)
    if param_values in seen:
        mlflow.delete_run(run["run_id"])
        print(f"Duplicate run found: {run['run_id']}")
        i += 1
    else:
        seen.add(param_values)

In [8]:
i

0

In [9]:
runs_df.shape

(3, 20)

In [10]:
seen

{(63.773140704273295, 103.47374243926491),
 (267.45025628793854, 276.51421901069017),
 (267.96269169866514, 277.2464916761085)}

In [11]:
runs_df = mlflow.search_runs(experiment_ids=[experiment_id])
runs_df.shape

(3, 20)